# Collecting Rent Offers from Daft.ie website

Install dependencies

In [1]:
#%pip install bs4 lxml requests-cache tqdm > /dev/null

In [2]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests_cache
import warnings
import bs4
from tqdm import tqdm
import re


warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
daft_base_url = 'https://www.daft.ie'

In [4]:
session = requests_cache.CachedSession(
  '.daft_cache',
  use_cache_dir=True,
  cache_control=False,
  expire_after=datetime.timedelta(days=30),
  allowable_methods=['GET'],
  allowable_codes=[200],
  stale_if_error=True
)

Get frontpage with all listings

In [5]:
def get_page(url):
    r = session.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    data_script = soup.find('script', id='__NEXT_DATA__', type='application/json')
    if data_script:
        return json.loads(data_script.text)
    return None


def get_listing_page(page_number):
    url = f'{daft_base_url}/property-for-rent/ireland?from={page_number*20}&pageSize=20'
    return get_page(url)

In [6]:
first_page_data = get_listing_page(0)
total_pages = first_page_data['props']['pageProps']['paging']['totalPages']
total_pages

40

In [7]:
data = {
    0: first_page_data
}

In [8]:
for i in tqdm(range(1, total_pages)):
    data[i] = get_listing_page(i)

100%|██████████| 39/39 [00:03<00:00, 10.24it/s]


In [9]:
df = pd.DataFrame()

for item in data.values():
    if 'listings' not in item['props']['pageProps']:
        continue
    for listing in item['props']['pageProps']['listings']:
        df = df.append(listing['listing'], ignore_index=True)

In [10]:
df = df.convert_dtypes()
df = df.set_index('id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 798 entries, 3581863 to 3175844
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              798 non-null    string 
 1   seoTitle           798 non-null    string 
 2   sections           798 non-null    object 
 3   saleType           798 non-null    object 
 4   featuredLevel      798 non-null    string 
 5   featuredLevelFull  798 non-null    string 
 6   publishDate        798 non-null    Int64  
 7   price              798 non-null    string 
 8   abbreviatedPrice   798 non-null    string 
 9   numBedrooms        760 non-null    string 
 10  propertyType       798 non-null    string 
 11  daftShortcode      798 non-null    string 
 12  seller             798 non-null    object 
 13  media              798 non-null    object 
 14  ber                694 non-null    object 
 15  platform           798 non-null    string 
 16  point              79

In [11]:
df_details = pd.DataFrame()

for index, values in tqdm(df.iterrows()):
    url = daft_base_url + values.seoFriendlyPath
    r = get_page(url)
    if not r:
        print(f'No response from {url}')
        continue

    page_props = r['props']['pageProps']

    def add_item(item):
        global df_details
        if 'listingViews' in page_props:
            item['listingViews'] = page_props['listingViews']
        else:
            item['listingViews'] = 0

        df_details = df_details.append(item, ignore_index=True)

    # page with only one listing
    if 'listing' in page_props:
        item = page_props['listing']
        add_item(item)

    # page with multiple listings
    elif 'listings' in page_props:
        for item in page_props['listings']:
            add_item(item)


798it [00:31, 25.27it/s]


In [12]:
df_details['id'] = df_details['id'].astype(df.index.dtype)

In [13]:
df_details = df_details.set_index('id')
df_details.head()

,title,seoTitle,sections,featuredLevel,featuredLevelFull,lastUpdateDate,price,numBedrooms,propertyType,daftShortcode,...,features,nonFormatted,listingViews,listing,savedAd,numBathrooms,priceHistory,propertyOverview,openViewings,label
id,,,,,,,,,,,,,,,,,,,,,
3581863,"The Benson, The Benson, Britain Quay, Dublin 2","The Benson, The Benson, Britain Quay, Dublin 2","[Property, Private Rental Sector, Apartments]",FEATURED,FEATURED_PLUS,14/09/2022,"From €3,300 per month",2 & 3 bed,Apartments,96577784,...,"[Open plan living / dining area , Floor to cei...","{'price': 3300, 'section': 'prs'}",100212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1442724,"Opus, 6 Hanover Quay, Hanover Quay, Dublin 2","Opus, 6 Hanover Quay, Hanover Quay, Dublin 2","[Property, Private Rental Sector, Apartments]",FEATURED,FEATURED_PLUS,14/09/2022,"From €6,222 per month",3 & 4 bed,Apartments,9161879,...,[Spacious and bright entrance lobby where you ...,"{'price': 6222, 'section': 'prs'}",369602,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446982,"Quayside Quarter, North Wall Quay, Dublin 1","Quayside Quarter, North Wall Quay, Dublin 1","[Property, Private Rental Sector, Apartments]",FEATURED,FEATURED,09/09/2022,"From €2,710 per month","1, 2 & 3 bed",Apartments,9162471,...,"[Concierge 24/7 and Resident Gym , Luxury wate...","{'price': 2710, 'section': 'prs'}",1326024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3523580,"Griffith Wood, Griffith Avenue, Drumcondra, Du...","Griffith Wood, Griffith Avenue, Drumcondra, Du...","[Property, Private Rental Sector, Apartments]",FEATURED,FEATURED,09/09/2022,"From €2,445 per month",2 & 3 bed,Apartments,96115505,...,[Brand new development],"{'price': 2445, 'section': 'prs'}",264646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3652525,"Hali, Cherrywood, Dublin 18, Cherrywood, Co. D...","Hali, Cherrywood, Dublin 18, Cherrywood, Co. D...","[Property, Private Rental Sector, Apartments]",FEATURED,FEATURED,07/09/2022,"From €1,660 per month","1, 2 & 3 bed",Apartments,97315531,...,"[Open plan living, Fully furnished, High spec ...","{'price': 1660, 'section': 'prs'}",8387,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
detail_only_columns = [c for c in df_details.columns.tolist() if c not in df.columns.tolist()]
detail_only_columns

['lastUpdateDate',
 'description',
 'primaryAreaId',
 'isInRepublicOfIreland',
 'facilities',
 'features',
 'nonFormatted',
 'listingViews',
 'listing',
 'savedAd',
 'propertyOverview']

Join new columns comming from detailed page.

In [15]:
df = df.join(df_details[detail_only_columns])

### Expand fields to create new features

In [16]:
def fix_case(col, prefix=''):
    # remove all spaces
    col = col.strip().replace(' ', '')

    # make sure that first letter is lowercase
    col = col[0].lower() + col[1:]

    # replace uppercase letters with _ + lowercase
    col = ''.join([
        '_' + l.lower() if l.isupper() else l for l in col])

    fixed_name = prefix + col

    # simple fix to avoid double underscores in column names
    return fixed_name.replace('__', '_')

In [17]:
def standarize_columns(df, orig):
    cols = df.columns.tolist()
    new_columns = [fix_case(orig + x) for x in cols]
    df.columns = new_columns
    return df

In [18]:
def join(df1, df2):
    if df2.shape[0] > df1.shape[0]:
        raise Exception('Right data frame is bigger than left')
    for col in df2.columns.tolist():
        df1[col] = df2[col]
    return df1

In [19]:
def prepare(df, col):
    df.columns = [fix_case(x, col + '_') for x in df.columns.tolist()]
    return df.dropna(axis=1, how='any', thresh=df.shape[0]*.3)

In [20]:
df.columns = [fix_case(c) for c in df.columns.tolist()]

**section**

In [21]:
%%time
all_sections = df['sections'].explode().unique().tolist()
all_sections

CPU times: user 1.85 ms, sys: 926 µs, total: 2.77 ms
Wall time: 2.05 ms


['Property',
 'Residential',
 'Apartment',
 'Private Rental Sector',
 'Apartments',
 'Studio Apartment',
 'House',
 'Flat']

In [22]:
def row_to_column(df, col, options):
    df_options = pd.DataFrame(columns=options, index=df.index)
    for option in options:
        df_options[option] = df[col].apply(lambda p: 1 if option in p else 0)
    return standarize_columns(df_options, col)

In [23]:
%%time

df_sections = row_to_column(df, 'sections', all_sections)
df_sections.head()

CPU times: user 8.16 ms, sys: 1.01 ms, total: 9.17 ms
Wall time: 8.48 ms


,sections_property,sections_residential,sections_apartment,sections_private_rental_sector,sections_apartments,sections_studio_apartment,sections_house,sections_flat
id,,,,,,,,
152700,1,1,1,0,0,0,0,0
1442724,1,0,0,1,1,0,0,0
1443907,1,0,0,1,1,0,0,0
1446982,1,0,0,1,1,0,0,0
1527608,1,0,0,1,1,0,0,0


In [24]:
df = df.join(df_sections)
df.head()

,title,seo_title,sections,sale_type,featured_level,featured_level_full,publish_date,price,abbreviated_price,num_bedrooms,...,saved_ad,property_overview,sections_property,sections_residential,sections_apartment,sections_private_rental_sector,sections_apartments,sections_studio_apartment,sections_house,sections_flat
id,,,,,,,,,,,,,,,,,,,,,
152700,"Serviced Apartments in Malahide, Malahide, Co....","Serviced Apartments in Malahide, Malahide, Co....","[Property, Residential, Apartment]",[To Let],FEATURED,FEATURED,1663152762654,€980 per week,€980,2 Bed,...,NaN,"[{'label': 'Double Bedroom', 'text': '2'}, {'l...",1,1,1,0,0,0,0,0
1442724,"Opus, 6 Hanover Quay, Hanover Quay, Dublin 2","Opus, 6 Hanover Quay, Hanover Quay, Dublin 2","[Property, Private Rental Sector, Apartments]",[To Let],FEATURED,FEATURED_PLUS,1663166461000,"From €6,222 per month","€6,220+",3 & 4 bed,...,NaN,NaN,1,0,0,1,1,0,0,0
1443907,"Capital Dock Residence, Grand Canal , Dublin 2","Capital Dock Residence, Grand Canal , Dublin 2","[Property, Private Rental Sector, Apartments]",[To Let],PREMIUM,PREMIUM,1660298367000,"From €10,440 per month","€10,440",2 bed,...,NaN,NaN,1,0,0,1,1,0,0,0
1446982,"Quayside Quarter, North Wall Quay, Dublin 1","Quayside Quarter, North Wall Quay, Dublin 1","[Property, Private Rental Sector, Apartments]",[To Let],FEATURED,FEATURED,1662738980000,"From €2,710 per month","€2,710+","1, 2 & 3 bed",...,NaN,NaN,1,0,0,1,1,0,0,0
1527608,"Node Living, 25 pembroke street upper, Dublin 2","Node Living, 25 pembroke street upper, Dublin 2","[Property, Private Rental Sector, Apartments]",[To Let],FEATURED,FEATURED,1660573247000,"From €4,039 per month","€4,040",2 bed,...,NaN,NaN,1,0,0,1,1,0,0,0


**saleType**

In [25]:
all_sale_types = df['sale_type'].explode().unique()
all_sale_types

array(['To Let'], dtype=object)

In [26]:
%%time

df_sale_type = row_to_column(df, 'sale_type', all_sale_types)
df_sale_type.head()

CPU times: user 2.64 ms, sys: 524 µs, total: 3.16 ms
Wall time: 2.84 ms


,sale_type_to_let
id,
152700,1
1442724,1
1443907,1
1446982,1
1527608,1


In [27]:
df = join(df, df_sale_type)

**publish_date**

In [28]:
df['publish_date'] = pd.to_datetime(df['publish_date'], unit='ms')

**price**

In [29]:
df_price = pd.DataFrame(index=df.index)
df_price['is_price_weekly'] = df['price'].apply(lambda s: 1 if 'week' in s else 0)
df_price['is_price_monthly'] = df['price'].apply(lambda s: 1 if 'month' in s else 0)
df_price['is_price_on_application'] = df['price'].apply(lambda s: 1 if 'Price on Application' in s else 0)

In [30]:
%%time

euro_cost = re.compile(r"€(\d*),?(\d+)")

df_price['price_as_int'] = df['price'].apply(
    lambda s: int(''.join(euro_cost.findall(s)[0])) if euro_cost.search(s) else 0)

CPU times: user 2.22 ms, sys: 67 µs, total: 2.29 ms
Wall time: 2.26 ms


In [31]:
df_price['monthly_price'] = (
    df_price['price_as_int'] * (4 * df_price['is_price_weekly'])) + (
        df_price['price_as_int'] * df_price['is_price_monthly'])

In [32]:
df = join(df, df_price)

**num_bedrooms & num_bathrooms**

In [33]:
numbers = re.compile(r"(\d+)\D+")

def extract_numbers(col):
    df_temp = pd.DataFrame(index=df.index)
    return df.fillna('')[col].apply(numbers.findall)

In [34]:
df['num_bedrooms_as_list'] = extract_numbers('num_bedrooms')

In [35]:
df['num_bathrooms_as_list'] = extract_numbers('num_bathrooms')

**seller**

In [36]:
%%time
df_seller = pd.DataFrame(columns=df['seller'].iloc[0].keys(), index=df.index)
for index, item in zip(df['seller'].index, df['seller'].tolist()):
    for col, value in item.items():
        df_seller.loc[index, col] = value

CPU times: user 617 ms, sys: 10.5 ms, total: 627 ms
Wall time: 642 ms


In [37]:
df_seller.columns = [fix_case(c, 'seller_') for c in df_seller.columns]

In [38]:
df = join(df, df_seller)

**media**

In [39]:
from collections import defaultdict

def iterate_over_struct(fn, index, item, root):
    item_type = type(item)
    if item_type == dict:
        for key, value in item.items():
            iterate_over_struct(fn, index, value, root + [key])
    elif item_type == list:
        for index_value, value in enumerate(item):
            iterate_over_struct(fn, index, value, root + [index_value])
    else:
        path = [str(r) for r in root if r]
        col = '_'.join(path)
        col = fix_case(col)
        fn(index, col, item)

def make_flat(df, col):
    final = defaultdict(dict)

    def add_to_list(index, col, value):
        final[index][col] = value

    for index, line in zip(df.index.tolist(), df[col]):
        iterate_over_struct(add_to_list, index, line, [col])

    df_new = pd.DataFrame(final.values(), index=final.keys())
    return df_new

In [40]:
df_medias = make_flat(df, 'media')
df_medias.head()

,media_images_size720x480,media_images_size600x600,media_images_size400x300,media_images_size360x240,media_images_size300x200,media_images_size320x280,media_images_size72x52,media_images_size680x392,media_images_1_size720x480,media_images_1_size600x600,...,media_images_3_size680x392,media_total_images,media_has_video,media_has_virtual_tour,media_has_brochure,media_images_caption,media_images_2_caption,media_images_3_caption,media_images_1_caption,media_brochure_url
152700,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,30,True,False,False,NaN,NaN,NaN,NaN,NaN
1442724,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,53,False,False,False,NaN,NaN,NaN,NaN,NaN
1443907,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,25,True,False,False,NaN,NaN,NaN,NaN,NaN
1446982,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,38,False,False,False,NaN,NaN,NaN,NaN,NaN
1527608,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,21,True,False,False,Apartment's private living room,NaN,NaN,NaN,NaN


In [41]:
df = join(df, df_medias)

**ber**

In [42]:
df_ber = pd.DataFrame([x if type(x) == dict else {} for x in df['ber'].tolist()], index=df.index)
df_ber.head()

,rating,code,epi
id,,,
152700,SI_666,NaN,NaN
1442724,A2,NaN,NaN
1443907,A2,NaN,NaN
1446982,NA,NaN,NaN
1527608,C2,NaN,NaN


In [43]:
df_ber.columns = [fix_case(x, 'ber_') for x in df_ber.columns.tolist()]

In [44]:
df_ber.dropna(axis=1, how='any', thresh=df_ber.shape[0]*.3, inplace=True)

In [45]:
df_ber.head()

,ber_rating
id,
152700,SI_666
1442724,A2
1443907,A2
1446982,NA
1527608,C2


In [46]:
df = join(df, df_ber)

**point**

In [47]:
df_point = pd.DataFrame(index=df.index)

In [48]:
df_point['point_type'] = df['point'].apply(lambda p: p['type'])

In [49]:
df_point['point_lat'] = df['point'].apply(lambda p: p['coordinates'][0] if p and 'coordinates' in p else np.nan)
df_point['point_long'] = df['point'].apply(lambda p: p['coordinates'][1] if p and 'coordinates' in p else np.nan)

In [50]:
df = join(df, df_point)

**page_branding**

In [51]:
df_page_branding = make_flat(df, 'page_branding')
df_page_branding.head()

,page_branding_standard_logo,page_branding_square_logo,page_branding_background_colour,page_branding_square_logos,page_branding_rectangle_logo,page_branding
152700,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,#f4f4f4,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN
1442724,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,#fee775,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN
1443907,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,#054573,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN
1446982,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN,#ffffff,None,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN
1527608,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,#c6e4d2,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,https://media.daft.ie/eyJidWNrZXQiOiJtZWRpYW1h...,NaN


In [52]:
df = join(df, df_page_branding)

**last_update_date**

In [53]:
df['last_update_date'] = pd.to_datetime(df['last_update_date'])

**facilities**

In [54]:
df_facilities = pd.DataFrame(index=df.index)
df_facilities['facilities_as_list'] = df['facilities'].apply(lambda x: [f['key'] for f in x] if isinstance(x, list) else [])
df_facilities.head()

,facilities_as_list
id,
152700,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W..."
1442724,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W..."
1443907,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W..."
1446982,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W..."
1527608,"[CENTRAL_HEATING, WASHING_MACHINE, DRYER, DISH..."


In [55]:
all_facilities = []
for list_of_facilities in df_facilities['facilities_as_list']:
    for facility in list_of_facilities:
        if facility not in all_facilities:
            all_facilities.append(facility)
all_facilities

['PARKING',
 'CENTRAL_HEATING',
 'CABLE_TELEVISION',
 'WASHING_MACHINE',
 'DRYER',
 'DISHWASHER',
 'MICROWAVE',
 'INTERNET',
 'GARDEN_PATIO_BALCONY',
 'SERVICED_PROPERTY',
 'PETS_ALLOWED',
 'CONCIERGE',
 'GYM',
 'COMMUNITY_EVENTS',
 'CO_WORKING',
 'WHEELCHAIR_ACCESS',
 'RESIDENTS_LOUNGE',
 'ENTERTAINMENT',
 'LAUNDRY',
 'ALARM',
 'SMOKING']

In [56]:
for facility in all_facilities:
    df_facilities['facility_' + facility.lower()] = df_facilities['facilities_as_list'].apply(lambda f: 1 if facility in f else 0)
df_facilities.head()

,facilities_as_list,facility_parking,facility_central_heating,facility_cable_television,facility_washing_machine,facility_dryer,facility_dishwasher,facility_microwave,facility_internet,facility_garden_patio_balcony,...,facility_concierge,facility_gym,facility_community_events,facility_co_working,facility_wheelchair_access,facility_residents_lounge,facility_entertainment,facility_laundry,facility_alarm,facility_smoking
id,,,,,,,,,,,,,,,,,,,,,
152700,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W...",1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1442724,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,0,0,0
1443907,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,0
1446982,"[PARKING, CENTRAL_HEATING, CABLE_TELEVISION, W...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,0,0
1527608,"[CENTRAL_HEATING, WASHING_MACHINE, DRYER, DISH...",0,1,0,1,1,1,1,1,1,...,1,0,1,1,0,1,0,0,0,0


In [57]:
df = join(df, df_facilities)

**non_formatted**

In [58]:
df_non_formatted = make_flat(df, 'non_formatted')
df_non_formatted.head()

,non_formatted_beds,non_formatted_price,non_formatted_section,non_formatted
152700,2.0,980.0,residential-to-rent,NaN
1442724,NaN,6222.0,prs,NaN
1443907,NaN,10440.0,prs,NaN
1446982,NaN,2710.0,prs,NaN
1527608,NaN,4039.0,prs,NaN


In [59]:
df = join(df, df_non_formatted)

**property_overview**

In [60]:
df_property_overview = pd.DataFrame(index=df.index)
df_property_overview['property_overview'] = df['property_overview'].apply(lambda x: {k['label']: k['text'] for k in x} if isinstance(x, list) else np.nan)

In [61]:
df_property_overview =  make_flat(df_property_overview, 'property_overview').drop(columns=['property_overview'])

In [62]:
df = join(df, df_property_overview)

### Drop columns with no sufficient data

Drop columns with no values

In [63]:
df = df.dropna(axis=1, how='all')

Drop columns with less than 30% of data

In [64]:
df.dropna(axis=1, how='any', thresh=df.shape[0]*.3, inplace=True)

Convert dtypes

In [68]:
df = df.convert_dtypes()

In [71]:
df.to_csv('data/daft.csv')